In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import fastai
from fastai.train import Learner
from fastai.train import DataBunch
from fastai.callbacks import *
from fastai.basic_data import DatasetType
import fastprogress
from fastprogress import force_console_behavior
import numpy as np
from pprint import pprint
import pandas as pd
import os
import time

import gc
import random
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F

import os
print(os.listdir("../input/"))

from nltk.tokenize.treebank import TreebankWordTokenizer

tqdm.pandas()

Using TensorFlow backend.


['pickleds', 'gpt2-inferece-w-identity', 'bert-inference-w-identityseed5060', 'modelos-entrenados-toxic', 'ppbert', 'jigsaw-unintended-bias-in-toxicity-classification', 'bert-inference-w-identity-1', 'gpt2-pytorch', 'bert-inference-w-identity', 'bert-inference-w-identity-seed6789', 'bert-inference-w-identity-2', 'bert-pretrained-models', 'gpt2-models']


In [3]:
print(os.listdir("../input/pickleds/"))

['pickled-crawl300d2m-for-kernel-competitions', 'pickled-glove840b300d-for-10sec-loading']


In [4]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    return np.array(all_tokens)

In [5]:
# Any results you write to the current directory are saved as output.
modelos_path = '../input/modelos-entrenados-toxic/'

def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

#CRAWL_EMBEDDING_PATH = '../input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl'
#GLOVE_EMBEDDING_PATH = '../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
CRAWL_EMBEDDING_PATH = '../input/pickleds/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl'
GLOVE_EMBEDDING_PATH = '../input/pickleds/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'


In [6]:
NUM_MODELS = 2
LSTM_UNITS = 128 #128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
MAX_LEN = 220 #220
batch_s = 512
EPOCHS = 5
learning_rate = 0.001
lambda_model = 0.6
spatial_dropout = 0.3 #0.3 ///0.01, 0.5 baja
EMBED_SIZE = 600

symbols_to_isolate = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
symbols_to_delete = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'


In [7]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr
    
def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((max_features + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        if i <= max_features:
            try:
                embedding_matrix[i] = embedding_index[word]
            except KeyError:
                try:
                    embedding_matrix[i] = embedding_index[word.lower()]
                except KeyError:
                    try:
                        embedding_matrix[i] = embedding_index[word.title()]
                    except KeyError:
                        unknown_words.append(word)
    return embedding_matrix, unknown_words

In [8]:
class EMB(nn.Module):
    def __init__(self, embedding_matrix):
        super(EMB, self).__init__()
        embed_size = embedding_matrix.shape[1]
        
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(spatial_dropout)
        
    def forward(self, x, lengths=None):
        h_embedding = self.embedding(x.long())
        h_embedding = self.embedding_dropout(h_embedding)
        
        return out
    
class RoM(nn.Module):
    def __init__(self, num_aux_targets):
        super(RoM, self).__init__()
        embed_size = EMBED_SIZE
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)
    
        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        
        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
        self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)
        
    def forward(self, x, lengths=None):
        
        h_lstm1, _ = self.lstm1(x)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        # global average pooling
        avg_pool = torch.mean(h_lstm2, 1)
        # global max pooling
        max_pool, _ = torch.max(h_lstm2, 1)
        
        h_conc = torch.cat((max_pool, avg_pool), 1)
        h_conc_linear1  = F.relu(self.linear1(h_conc))
        h_conc_linear2  = F.relu(self.linear2(h_conc))
        
        hidden = h_conc + h_conc_linear1 + h_conc_linear2
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x
    
class NeuralNet(nn.Module):
    def __init__(self, embedding_matrix, num_aux_targets):
        super(NeuralNet, self).__init__()
        embed_size = embedding_matrix.shape[1]
        
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(spatial_dropout)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)
    
        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        
        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
        self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)
        
    def forward(self, x, lengths=None):
        h_embedding = self.embedding(x.long())
        h_embedding = self.embedding_dropout(h_embedding)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        # global average pooling
        avg_pool = torch.mean(h_lstm2, 1)
        # global max pooling
        max_pool, _ = torch.max(h_lstm2, 1)
        
        h_conc = torch.cat((max_pool, avg_pool), 1)
        h_conc_linear1  = F.relu(self.linear1(h_conc))
        h_conc_linear2  = F.relu(self.linear2(h_conc))
        
        hidden = h_conc + h_conc_linear1 + h_conc_linear2
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out

In [10]:
tokenizer = TreebankWordTokenizer()


isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}


def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x

def handle_contractions(x):
    x = tokenizer.tokenize(x)
    return x

def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def preprocess(x):
    x = handle_punctuation(x)#movido a abajo
    x = handle_contractions(x)#movido a arriba 
    x = fix_quote(x)
    return x

In [11]:
test_df = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
#test_df = test_df.loc[:100, :] #++
IDS_TEST = test_df['id']

test_df['comment_text'] = test_df['comment_text'].astype(str) 

REDES
---

In [12]:
lstm_test = test_df['comment_text'].progress_apply(lambda x:preprocess(x))

max_features = 327576

#################################################################################################### 
tokenizer = text.Tokenizer(num_words = max_features, filters='',lower=False)
#3. In the inference kernel build the embedding matrix with only the words of the test set.
tokenizer.fit_on_texts(list(lstm_test)) 

crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH)
print('n unknown words (crawl): ', len(unknown_words_crawl))

glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH)
print('n unknown words (glove): ', len(unknown_words_glove))

max_features = max_features or len(tokenizer.word_index) + 1
max_features

embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)
embedding_matrix.shape

del crawl_matrix
del glove_matrix
gc.collect()

batch_size = batch_s
x_test = tokenizer.texts_to_sequences(lstm_test)

#maxlen = lengths.max() 
maxlen = 300
test_lengths = torch.from_numpy(np.array([len(x) for x in x_test]))
x_test_padded = torch.from_numpy(sequence.pad_sequences(x_test, maxlen=maxlen))


test_dataset = data.TensorDataset(x_test_padded, test_lengths)
#test_collator = SequenceBucketCollator(lambda lenghts: lenghts.max(), sequence_index=0, length_index=1)
#test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_collator)

gc.collect()


n unknown words (crawl):  14055
n unknown words (glove):  14375


0

In [13]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#TODO cargar modelos y calcular medias de las predicciones
# meter como primera capa el embedding weigth
all_test_preds = []

cont = 0
for filename in os.listdir(modelos_path):
    
    if cont > 6:
        break
    
    print('Modelo: '+filename)
    #creamos/cargamos los modelos separados
    embd_model = EMB(embedding_matrix).cuda()
    ron_model = RoM(6).cuda()
    ron_model.load_state_dict(torch.load(modelos_path + filename))

    # y juntamos
    state_dict={}
    for key, value in embd_model.state_dict().items():
        state_dict[key] = value
    
    for key, value in ron_model.state_dict().items():
        state_dict[key] = value
    
    del embd_model, ron_model
    
    model = NeuralNet(embedding_matrix, 6).cuda()
    model.load_state_dict(state_dict)
    
    #++----------
    for param in model.parameters():
        param.requires_grad = False
    model.eval() #++
    #++----------
    
    print('Modelo preparado y cargado.')
    test_preds = np.zeros((len(test_dataset), 7))
    start = time.time()
    for i, x_batch in enumerate(test_loader):
            X = x_batch[0].cuda()
            y_pred = sigmoid(model(X).detach().cpu().numpy())
            test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred

    end = time.time()
    print('Tiempo prediccion:', end-start)
    all_test_preds.append(test_preds)
    cont+=1
    
    gc.collect()
    
submission_lstm = pd.DataFrame.from_dict({
    'id': IDS_TEST,
    'prediction': np.mean(all_test_preds, axis=0)[:, 0]
})

Modelo: model_3.pth
Modelo preparado y cargado.
Tiempo prediccion: 15.089172840118408
Modelo: model_5.pth
Modelo preparado y cargado.
Tiempo prediccion: 15.061607122421265
Modelo: model_4.pth
Modelo preparado y cargado.
Tiempo prediccion: 15.047003269195557
Modelo: model_2.pth
Modelo preparado y cargado.
Tiempo prediccion: 15.082767248153687
Modelo: model_1.pth
Modelo preparado y cargado.
Tiempo prediccion: 15.044901371002197
Modelo: model_0.pth
Modelo preparado y cargado.
Tiempo prediccion: 15.060503482818604


In [14]:
warnings.filterwarnings(action='once')
device = torch.device('cuda')

GPT2
---

In [15]:
'''
! pip install ../input/gpt2-pytorch/gpt2-pytorch/pytorch-pretrained-BERT-master
package_dir = "../input/gpt2-pytorch/gpt2-pytorch/pytorch-pretrained-BERT-master"
sys.path.append(package_dir)

from pytorch_pretrained_bert import GPT2Config
from pytorch_pretrained_bert import GPT2Tokenizer
from pytorch_pretrained_bert import GPT2ClassificationHeadModel
'''

'\n! pip install ../input/gpt2-pytorch/gpt2-pytorch/pytorch-pretrained-BERT-master\npackage_dir = "../input/gpt2-pytorch/gpt2-pytorch/pytorch-pretrained-BERT-master"\nsys.path.append(package_dir)\n\nfrom pytorch_pretrained_bert import GPT2Config\nfrom pytorch_pretrained_bert import GPT2Tokenizer\nfrom pytorch_pretrained_bert import GPT2ClassificationHeadModel\n'

In [16]:
def convert_lines2(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    return np.array(all_tokens)

In [17]:
'''
%%time
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
BATCH_SIZE_gpt2 = 16
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/' #base ->10xx es large

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

gpt2_config = GPT2Config('../input/gpt2-inferece-w-identity/results/'+'config.json')
tokenizer = GPT2Tokenizer.from_pretrained('../input/gpt2-models/')#++

X_test_gpt2 = convert_lines2(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)

model = GPT2ClassificationHeadModel(gpt2_config,n_class=7)
model.load_state_dict(torch.load("../input/gpt2-inferece-w-identity/results/gpt2_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

test_preds_gpt2 = np.zeros((len(X_test_gpt2)))
test_gpt2 = torch.utils.data.TensorDataset(torch.tensor(X_test_gpt2, dtype=torch.long))
test_loader_gpt2 = torch.utils.data.DataLoader(test_gpt2, batch_size=BATCH_SIZE_gpt2, shuffle=False)
tk0 = tqdm(test_loader_gpt2)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device))
    test_preds_gpt2[i * BATCH_SIZE_gpt2:(i + 1) * BATCH_SIZE_gpt2] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_gpt2 = torch.sigmoid(torch.tensor(test_preds_gpt2)).numpy().ravel()

submission_gpt2 = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': test_pred_gpt2
})
'''

'\n%%time\nMAX_SEQUENCE_LENGTH = 220\nSEED = 1234\nBATCH_SIZE_gpt2 = 16\nBERT_MODEL_PATH = \'../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/\' #base ->10xx es large\n\nnp.random.seed(SEED)\ntorch.manual_seed(SEED)\ntorch.cuda.manual_seed(SEED)\ntorch.backends.cudnn.deterministic = True\n\ngpt2_config = GPT2Config(\'../input/gpt2-inferece-w-identity/results/\'+\'config.json\')\ntokenizer = GPT2Tokenizer.from_pretrained(\'../input/gpt2-models/\')#++\n\nX_test_gpt2 = convert_lines2(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)\n\nmodel = GPT2ClassificationHeadModel(gpt2_config,n_class=7)\nmodel.load_state_dict(torch.load("../input/gpt2-inferece-w-identity/results/gpt2_pytorch.bin"))\nmodel.to(device)\nfor param in model.parameters():\n    param.requires_grad = False\nmodel.eval()\n\ntest_preds_gpt2 = np.zeros((len(X_test_gpt2)))\ntest_gpt2 = torch.utils.data.TensorDataset(torch.tensor(X_test_gpt2, dtype=torch.long))\ntest_l

In [18]:
gc.collect()

0

BERT
---

In [19]:
#BERT
import sys
package_dir = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
sys.path.append(package_dir)

! pip install ../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT

Processing /kaggle/input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT
  Stored in directory: /tmp/.cache/pip/wheels/bf/86/9a/36f2f25ad22af9ff95c8b53880174e7ed7422255a0054a7a8e
Successfully built pytorch-pretrained-bert
ERROR: allennlp 0.8.4 requires awscli>=1.11.91, which is not installed.
ERROR: allennlp 0.8.4 requires flaky, which is not installed.
ERROR: allennlp 0.8.4 requires responses>=0.7, which is not installed.
  Found existing installation: pytorch-pretrained-bert 0.6.2
    Uninstalling pytorch-pretrained-bert-0.6.2:
      Successfully uninstalled pytorch-pretrained-bert-0.6.2


In [20]:
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
from pytorch_pretrained_bert import BertConfig



In [21]:
print(os.listdir('../input/bert-inference-w-identity/results/bert-inference-w-identity'))

['bert_pytorch.bin', 'custom.css', '__results__.html', 'pytorch_model.bin', 'bert_config.json']


In [22]:
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
BATCH_SIZE = 32
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/' #base ->10xx es large

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

bert_config = BertConfig('../input/bert-inference-w-identity/results/bert-inference-w-identity/bert_config.json')
bert_tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

In [23]:
X_test = convert_lines(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, bert_tokenizer)

In [24]:
model = BertForSequenceClassification(bert_config, num_labels=7)
model.load_state_dict(torch.load("../input/bert-inference-w-identity/results/bert-inference-w-identity/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediat

In [25]:
test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

bert_test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [26]:
bert_config2 = BertConfig('../input/bert-inference-w-identity-2/results/bert_config.json')

model2 = BertForSequenceClassification(bert_config2, num_labels=7)
#model2.load_state_dict(torch.load("../input/bert-inference-w-identity-1/results/bert_pytorch_3.bin"))
model2.load_state_dict(torch.load("../input/bert-inference-w-identity-2/results/bert_pytorch_4.bin"))
model2.to(device)
for param in model2.parameters():
    param.requires_grad = False
model2.eval()

test_preds2 = np.zeros((len(X_test)))
for i, (x_batch,) in enumerate(tk0):
    pred = model2(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds2[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred2 = torch.sigmoid(torch.tensor(test_preds2)).numpy().ravel()



In [27]:
bert_config3 = BertConfig('../input/bert-inference-w-identity-1/results/bert_config.json')

model3 = BertForSequenceClassification(bert_config3, num_labels=7)
model3.load_state_dict(torch.load("../input/bert-inference-w-identity-1/results/bert_pytorch_3.bin"))
#model2.load_state_dict(torch.load("../input/bert-inference-w-identity-2/results/bert_pytorch_4.bin"))
model3.to(device)
for param in model3.parameters():
    param.requires_grad = False
model3.eval()

test_preds3 = np.zeros((len(X_test)))
for i, (x_batch,) in enumerate(tk0):
    pred = model3(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds3[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred3 = torch.sigmoid(torch.tensor(test_preds3)).numpy().ravel()

In [28]:
bert_config4 = BertConfig('../input/bert-inference-w-identity-seed6789/results/bert_config.json')

model4 = BertForSequenceClassification(bert_config4, num_labels=7)
model4.load_state_dict(torch.load("../input/bert-inference-w-identity-seed6789/results/bert_pytorch.bin"))
#model2.load_state_dict(torch.load("../input/bert-inference-w-identity-2/results/bert_pytorch_4.bin"))
model4.to(device)
for param in model4.parameters():
    param.requires_grad = False
model4.eval()

test_preds4 = np.zeros((len(X_test)))
for i, (x_batch,) in enumerate(tk0):
    pred = model4(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds4[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred4 = torch.sigmoid(torch.tensor(test_preds4)).numpy().ravel()

In [29]:
bert_config5 = BertConfig('../input/bert-inference-w-identityseed5060/results/bert_config.json')

model5 = BertForSequenceClassification(bert_config5, num_labels=7)
model5.load_state_dict(torch.load("../input/bert-inference-w-identityseed5060/results/bert_pytorch.bin"))
#model2.load_state_dict(torch.load("../input/bert-inference-w-identity-2/results/bert_pytorch_4.bin"))
model5.to(device)
for param in model5.parameters():
    param.requires_grad = False
model5.eval()

test_preds5 = np.zeros((len(X_test)))
for i, (x_batch,) in enumerate(tk0):
    pred = model5(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds5[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred5 = torch.sigmoid(torch.tensor(test_preds5)).numpy().ravel()

Media Berts
---

In [30]:
bert_test_pred = (bert_test_pred + test_pred2 + test_pred3 + test_pred4 + test_pred5) / 5
#bert_test_pred = (test_pred2 + test_pred3 + test_pred4) / 3

In [31]:
submission_bert = pd.DataFrame.from_dict({
    'id': IDS_TEST,
    'prediction': bert_test_pred
})
submission_bert2 = pd.DataFrame.from_dict({
    'id': IDS_TEST,
    'prediction': test_pred2
})
submission_bert3 = pd.DataFrame.from_dict({
    'id': IDS_TEST,
    'prediction': test_pred3
})
submission_bert4 = pd.DataFrame.from_dict({
    'id': IDS_TEST,
    'prediction': test_pred4
})
submission_bert5 = pd.DataFrame.from_dict({
    'id': IDS_TEST,
    'prediction': test_pred5
})

In [32]:
gc.collect()

416

JUNTAMOS TODO
---

In [33]:
submission = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
submission['id'] = IDS_TEST
#submission['prediction'] = (submission_lstm['prediction'].apply(lambda x: x*0.4) + submission_bert['prediction'].apply(lambda x: x*0.6))
submission['prediction'] = (submission_lstm['prediction'].apply(lambda x: x*0.22172817) + submission_bert['prediction'].apply(lambda x: x*0.10849379) +
                           submission_bert2['prediction'].apply(lambda x: x*0.16600595) + submission_bert3['prediction'].apply(lambda x: x*0.09252766) +
                           submission_bert4['prediction'].apply(lambda x: x*0.07704189) + submission_bert5['prediction'].apply(lambda x: x*0.11969974))

submission.to_csv('submission.csv', index=False)